# Lost Luggage Distribution  Problem

## Objective and Prerequisites

In this example, you’ll learn how to use mathematical optimization to solve a vehicle routing problem with time windows, which involves helping a company figure out the minimum number of vans required to deliver pieces of lost or delayed baggage to their rightful owners and determining the optimal assignment of vans to customers.

This model is example 27 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 287-289 and 343-344.

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MUI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as an *academic user*.

## Problem Description

A small company with six vans has a contract with a number of airlines to pick up lost or delayed baggage, belonging to customers in the London area, from Heathrow airport at 6 p.m. each evening. The contract stipulates that each customer must have their baggage delivered by 8 p.m. The company requires a model to advise them what is the minimum number of vans they need to use and to which customers each van should deliver and in what order. There is no practical capacity limitation on each van. Each van can hold all baggage that needs to be delivered in a two-hour period. To solve this problem, we can formulate an optimization model that minimizes the number of vans that need to be used.



##  Model Formulation


### Sets and Indices

$i,j \in \text{Locations} \equiv L=\{0,1..(n-1)\}$: Set of locations where $0$ is the index for the single depot -Heathrow airport, and $n$ is the number of locations.

$k \in \text{Vans} \equiv  V=\{0..K-1\}$: Index and set of vans, where $K$ is the number of vans.

$S_k \in S  $: Tour of van $k$, i.e. subset of locations visited by the van.

### Parameters

$t_{i,j} \in \mathbb{R}^+$: Travel time from location $i$  to location $j$.

### Decision Variables

$x_{i,j,k} \in \{0,1 \}$: This binary variable is equal 1, if van $k$ visits and goes directly from location $i$ to location $j$, and zero otherwise.

$y_{i,k} \in \{0,1 \}$: This binary variable is equal 1, if van $k$ visits location $i$, and zero otherwise.

$z_{k} \in \{0,1 \}$: This binary variable is equal 1, if van $k \in \{1,2..K\}$ is used, and zero otherwise.

### Objective Function

**Number of vans**: Minimize number of vans used.

\begin{equation}
\text{Minimize} \quad \sum_{k = 1}^{K} z_k
\end{equation}

### Constraints

**Van utilization**: For all locations different from the depot, i.e. $i > 0$, if the location is visited by van $k$, then it is used.

\begin{equation}
y_{i,k} \leq z_{k} \quad \forall i \in L \setminus \{0\}, \; k \in V
\end{equation}

**Travel time**: No van travels for more than 120 min. Note that we do not consider the travel time to return to the depot.

\begin{equation}
\sum_{i \in L} \sum_{j \in L \setminus \{0\}} t_{i,j} \cdot x_{i,j,k} \leq 120 \quad \forall k \in  V
\end{equation}

**Visit all customers**:  Each customer location is visited by exactly one van.

\begin{equation}
\sum_{k \in V}  y_{i,k} = 1 \quad \forall i \in L \setminus \{0\}
\end{equation}

**Depot**: Heathrow is visited by every van used. (Note: to improve performance, we diverge from the book by disaggregating this constraint).

\begin{equation}
y_{0,k} = z_k \quad \forall k \in V
\end{equation}

**Arriving at a location**: If location $j$ is visited by van $k$, then the van is coming from another location $i$.

\begin{equation}
\sum_{i \in L}  x_{i,j,k} =  y_{j,k}  \quad \forall j \in L, \; k \in V
\end{equation}

**Leaving a location**: If van $k$ leaves location $j$, then the van is going to another location $i$.

\begin{equation}
\sum_{i \in L}  x_{j,i,k} = y_{j,k}  \quad \forall j \in L, \; k \in V
\end{equation}

**Breaking symmetry**: 

\begin{equation}
\sum_{i \in L}  y_{i,k} \geq \sum_{i \in L}  y_{i,k+1} \quad \forall k \in  \{0..K-1\}
\end{equation}

**Subtour elimination**: These constraints ensure that for each van route, there is no cycle. 

\begin{equation}
\sum_{(i,j) \in S_k}x_{i,j,k} \leq |S_k|-1 \quad \forall  k \in K, \;   S_k \subseteq L
\end{equation}



## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [1]:
import sys
import math
import random
from itertools import permutations
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

## Input data  
We define all the input data for the model. The user defines the number of locations, including the depot, and the number of vans. We randomly determine the coordinates of each location and then calculate the Euclidean distance between each pair of locations. We assume a speed of 60 km/hr, which is 1 km/min. Hence travel time is equal to the distance.

In [2]:
# number of locations, including the depot. The index of the depot is 0
n = 17
locations = [*range(n)]

# number of vans
K = 6
vans = [*range(K)]

# Create n random points
# Depot is located at (0,0) coordinates
random.seed(1)
points = [(0, 0)]
points += [(random.randint(0, 50), random.randint(0, 50)) for i in range(n-1)]

# Dictionary of Euclidean distance between each pair of points
# Assume a speed of 60 km/hr, which is 1 km/min. Hence travel time = distance
time = {(i, j):
        math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))
        for i in locations for j in locations if i != j}

## Model Deployment

We create a model and the variables. The decision variables determines the order in which each van visits a subset of custormers, which customer is visited by each van, and if a van is used or not.

In [3]:
m = gp.Model('lost_luggage_distribution.lp')

# Create variables: 

# x =1, if van  k  visits and goes directly from location  i  to location  j 
x = m.addVars(time.keys(), vans, vtype=GRB.BINARY, name='FromToBy')

# y = 1, if customer i is visited by van k
y = m.addVars(locations, vans, vtype=GRB.BINARY, name='visitBy')

# Number of vans used is a decision variable
z = m.addVars(vans, vtype=GRB.BINARY, name='used')

# Travel time per van
t = m.addVars(vans, ub=120, name='travelTime')

# Maximum travel time
s = m.addVar(name='maxTravelTime')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-29


## Constraints

For all locations different from depot, i.e. $i > 0$, if the location is visited by van $k$, then it is used.

In [4]:
# Van utilization constraint

visitCustomer = m.addConstrs((y[i,k] <= z[k]  for k in vans for i in locations if i > 0), name='visitCustomer' )

No van travels for more than 120 min. We make a small change from the original H.P. Williams version to introduce a slack variable for the travel time for each van, t[k].

In [5]:
# Travel time constraint
# Exclude the time to return to the depot

travelTime = m.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in vans), 
                          name='travelTimeConstr' )

Each customer location is visited by exactly one van

In [6]:
# Visit all customers
visitAll = m.addConstrs((y.sum(i,'*') == 1 for i in locations if i > 0), name='visitAll' )

Heathrow (depot) is visited by every van used.

In [7]:
# Depot constraint
depotConstr = m.addConstrs((y[0, k] == z[k] for k in vans), name='depotConstr' )

If location  j  is visited by van  k , then the van is coming from another location  i.

In [8]:
# Arriving at a customer location constraint
ArriveConstr = m.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr' )

 If van  k  leaves location  j , then the van is going to another location  i.

In [9]:
# Leaving a customer location constraint
LeaveConstr = m.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr' )

Breaking symmetry constraints.

In [10]:
breakSymm = m.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in vans if k>0), name='breakSymm' )

Relate the maximum travel time to the travel times of each van

In [11]:
maxTravelTime = m.addConstrs((t[k] <= s for k in vans), name='maxTravelTimeConstr')

# Alternately, as a general constraint:
# maxTravelTime = m.addConstr(s == gp.max_(t), name='maxTravelTimeConstr')

### Objective Function
We use two hierarchical objectives:
- First, minimize the number of vans used
- Then, minimize the maximum of the time limit constraints

In [12]:
m.ModelSense = GRB.MINIMIZE
m.setObjectiveN(z.sum(), 0, priority=1, name="Number of vans")
m.setObjectiveN(s, 1, priority=0, name="Travel time")

### Callback Definition
Subtour constraints prevent a van from visiting a set of destinations without starting or ending at the Heathrow depot. Because there are an exponential number of these constraints, we don't want to add them all to the model. Instead, we use a callback function to find violated subtour constraints and add them to the model as lazy constraints.

In [13]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour not containing depot (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

## Solve the model

In [14]:
# Verify model formulation

m.write('lost_luggage_distribution.lp')

# Run optimization engine
m._x = x
m.Params.LazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 339 rows, 1747 columns and 5492 nonzeros
Model fingerprint: 0x06e2dd4c
Variable types: 7 continuous, 1740 integer (1740 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 22 rows and 7 columns
Presolve time: 0.02s
Presolved: 317 rows and 174

## Analysis

The optimal route of each van used and the total lost luggage delivery time report follows.

In [15]:
# Print optimal routes
for k in vans:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"Route for van {k}: {i}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            print(f" -> {i}", end='')
            if i == 0:
                break
        print(f". Travel time: {round(t[k].X,2)} min")

print(f"Max travel time: {round(s.X,2)}")

Route for van 0: 0 -> 3 -> 16 -> 15 -> 2 -> 7 -> 14 -> 6 -> 5 -> 13 -> 0. Travel time: 105.42 min
Route for van 1: 0 -> 9 -> 8 -> 12 -> 1 -> 10 -> 4 -> 11 -> 0. Travel time: 105.16 min
Max travel time: 105.42


## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC